# 🎨 anime-character-generator: LoRA ファインチューニング

Stable Diffusion v1.5 を **アニメ・印象派風** に特化させるため、LoRA を使用したファインチューニングを Google Colab (T4 GPU) で実行します。

**実行時間:** 約 1-2 時間
**必要なもの:**
- Google Drive アカウント
- training_data/ フォルダ (660 MB)

## Step 1: 環境確認・GPU セットアップ

In [ ]:
# GPU の確認
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## Step 2: 依存パッケージのインストール

In [ ]:
# 必要なパッケージをインストール
!pip install -q diffusers transformers accelerate peft pillow torch tqdm safetensors

## Step 3: Google Drive にマウント

In [ ]:
from google.colab import drive

# Google Drive をマウント
drive.mount('/content/drive')
print("✅ Google Drive mounted")

## Step 4: データセットの準備

**事前準備 (このノートブック実行前):**
1. ローカルで `python scripts/download_danbooru.py --limit 60` を実行
2. 生成された `training_data/` フォルダを Google Drive にアップロード
3. 以下のセルを実行

In [ ]:
import os
from pathlib import Path

# Google Drive から training_data をコピー
drive_path = Path("/content/drive/MyDrive/training_data")
local_path = Path("/content/training_data")

if drive_path.exists():
    print(f"✅ Found: {drive_path}")
    # シンボリックリンク作成 (速度向上)
    if not local_path.exists():
        os.symlink(drive_path, local_path)
        print(f"✅ Symlink created: {local_path}")
    
    # データセット統計
    import json
    metadata_file = drive_path / "metadata.json"
    if metadata_file.exists():
        with open(metadata_file) as f:
            metadata = json.load(f)
        print(f"📊 Dataset: {len(metadata.get('training_data', []))} images")
else:
    print(f"❌ Not found: {drive_path}")
    print("\nPlease upload training_data/ to Google Drive")

## Step 5: train_lora.py のダウンロード

GitHub から最新の train_lora.py を取得

In [ ]:
# GitHub から train_lora.py をダウンロード
!wget -q https://raw.githubusercontent.com/Shion1124/anime-character-generator/master/train_lora.py -O /content/train_lora.py
print("✅ train_lora.py downloaded")

# ファイルサイズ確認
import os
file_size = os.path.getsize("/content/train_lora.py") / 1024
print(f"📊 File size: {file_size:.1f} KB")

## Step 6: LoRA トレーニング実行

**重要:** このセルは 1-2 時間 かかります。Colab の接続を保持してください。

In [ ]:
# トレーニング実行
!python /content/train_lora.py \
    --data_dir /content/training_data \
    --output_dir /content/lora_weights \
    --epochs 50 \
    --batch_size 1 \
    --learning_rate 1e-4 \
    --lora_rank 8 \
    --lora_alpha 32

## Step 7: 学習結果の確認

In [ ]:
import json
from pathlib import Path
import matplotlib.pyplot as plt

# 学習ログをロード
log_file = Path("/content/lora_weights/training_log.json")
if log_file.exists():
    with open(log_file) as f:
        training_log = json.load(f)
    
    # 学習曲線を表示
    losses = [h['loss'] for h in training_log['history']]
    epochs = list(range(1, len(losses) + 1))
    
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, losses, 'b-', linewidth=2, marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('LoRA Training Loss Curve')
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    plt.tight_layout()
    plt.savefig('/content/training_curve.png', dpi=100)
    plt.show()
    
    # 統計情報
    print(f"✅ Training completed!")
    print(f"📊 Total epochs: {len(losses)}")
    print(f"📉 Final loss: {losses[-1]:.6f}")
    print(f"📈 Initial loss: {losses[0]:.6f}")
    print(f"🎯 Loss improvement: {(losses[0] - losses[-1]) / losses[0] * 100:.1f}%")
else:
    print("⚠️ training_log.json not found")

## Step 8: LoRA 重みの確認

In [ ]:
import os
from pathlib import Path

lora_dir = Path("/content/lora_weights")
print("📁 Output files:")
print()

for file in sorted(lora_dir.glob("*")):
    if file.is_file():
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"  {file.name:40s} {size_mb:>8.2f} MB")

# メインの LoRA 重みファイル
lora_model = lora_dir / "anime-impressionist-lora.safetensors"
if lora_model.exists():
    size_mb = lora_model.stat().st_size / (1024 * 1024)
    print(f"\n✅ Main LoRA weights: {lora_model.name} ({size_mb:.2f} MB)")

## Step 9: Google Drive に結果を保存

In [ ]:
# 学習済み LoRA 重みを Google Drive に保存
import shutil
from pathlib import Path

lora_weights_dir = Path("/content/lora_weights")
drive_output_dir = Path("/content/drive/MyDrive/lora_weights")

# 出力ディレクトリ作成
drive_output_dir.mkdir(parents=True, exist_ok=True)

# ファイルをコピー
for file in lora_weights_dir.glob("*"):
    if file.is_file():
        destination = drive_output_dir / file.name
        shutil.copy2(file, destination)
        print(f"✅ Saved: {file.name}")

print(f"\n📁 Output saved to: {drive_output_dir}")
print("\n💾 Download instructions:")
print("1. Open Google Drive")
print("2. Navigate to /lora_weights/")
print("3. Download 'anime-impressionist-lora.safetensors'")
print("4. Place in: anime-character-generator/ root directory")

## Step 10: 推論テスト (オプション)

トレーニング済み LoRA モデルで推論テストを実行

In [ ]:
# LoRA を使用した推論テスト
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

print("⏳ Loading model...")

# パイプライン作成
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

# LoRA 重みロード
try:
    # PEFT を使用した LoRA ロード
    from peft import PeftModel
    # (LoRA ロード実装)
    print("✅ LoRA weights loaded")
except:
    print("⚠️ LoRA loading requires additional setup")

# テストプロンプト
prompt = "1girl, watercolor painting style, masterpiece, high quality, anime"
negative_prompt = "low quality, nsfw"

print(f"\n🎨 Generating: {prompt}...")

# 画像生成
with torch.no_grad():
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        height=512,
        width=512,
        num_inference_steps=20,
        guidance_scale=7.0
    ).images[0]

# 結果表示
image.save("/content/test_output.png")
print("✅ Image saved: test_output.png")
display(image)

## ✅ 完了！

**次のステップ:**
1. LoRA 重みをダウンロード
2. ローカルで `character_generator.py --use_lora=True` で推論
3. 結果を確認

**参考:**
- [GitHub リポジトリ](https://github.com/Shion1124/anime-character-generator)
- [HuggingFace モデル](https://huggingface.co/runwayml/stable-diffusion-v1-5)